In [100]:
import requests_async as r
from bs4 import BeautifulSoup
import re

In [101]:
processor = "Intel Core Ultra 7 265K"

res = await r.get(
    "https://www.cpubenchmark.net/cpu_list.php",
    headers={
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:134.0) Gecko/20100101 Firefox/134.0",
    }
)

print(res)
soup = BeautifulSoup(res.content, "html.parser")

cpu = soup.find(lambda tag: tag.name == "tr" and tag.find(lambda a: a.name == "a" and processor in a.text))
cpu_id = cpu.find("a").get("href")
cpu_id = re.search(r'id=\d+', cpu_id).group()
cpu_id = re.search(r'\d+', cpu_id).group()

print(cpu_id)

<Response [200 OK]>
6326


In [102]:
res = await r.get(
    "https://www.cpubenchmark.net/cpu.php?id=" + cpu_id,
    headers={
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:134.0) Gecko/20100101 Firefox/134.0",
    }
)

print(res)

<Response [200 OK]>


In [110]:
soup = BeautifulSoup(res.content, "html.parser")
desc_body = soup.find("div", class_="desc-body")



cores_threads = desc_body.find(lambda tag: tag.name == "p" and "Cores" in tag.text).text
if "total" in cores_threads.lower() or "perform" in cores_threads.lower() or "efficien" in cores_threads.lower():
    clockspeed = desc_body.find(lambda tag: tag.name == "p" and "GHz" in tag.text).text
    clockspeed = re.search(r'[\d\.]+[\s\w]*(Base)', clockspeed).group()

    cores = re.search(r'(\d+)\s+(Cores)', cores_threads).group()
    threads = re.search(r'(\d+)\s+(Threads)', cores_threads).group()
else:
    clockspeed = desc_body.find(lambda tag: tag.name == "p" and "Clockspeed" in tag.text).text
    cores = re.search(r'(.*Cores:\s*)(\d+)', cores_threads).group()
    threads = re.search(r'(Threads:\s*)(\d+)', cores_threads).group()

clockspeed = float(re.search(r'[\d\.]+', clockspeed).group())
cores = int(re.search(r'\d+', cores).group())
threads = int(re.search(r'\d+', threads).group())

print(clockspeed)
print(cores)
print(threads)

3.9
20
20
